In [41]:
import pandas as pd
import requests

In [42]:
# Setup and Imports
api_key = " 66848eff49cbd1.37105331"  # Replace with your EODHD API key
base_url = "https://eodhd.com/api/news"
limit = 100  # The number of articles per request (max 1000, default 50)
max_articles = 500  # Define the maximum number of articles to fetch


In [43]:
def fetch_news_data(ticker, start_date, end_date, limit, api_key, max_articles=500):
    offset = 0
    all_news_data = []

    while True:
        url = f"{base_url}?s={ticker}&from={start_date}&to={end_date}&limit={limit}&offset={offset}&api_token={api_key}&fmt=json"
        response = requests.get(url)

        if response.status_code == 200:
            news_data = response.json()
            if not news_data:
                break  # Break the loop if no more data is returned

            all_news_data.extend(news_data)

            # Increment the offset for the next batch of data
            offset += limit

            # Check if we've reached the max number of articles to fetch
            if len(all_news_data) >= max_articles:
                break  # Stop fetching more if we reach the maximum limit
        else:
            print(f"Failed to fetch data: {response.status_code}")
            break

    return all_news_data

In [44]:
def save_to_csv(data, ticker, folder='data'):
    """
    Save the news data to a CSV file.
    
    Parameters:
    data (pd.DataFrame): News Data
    ticker (str): Stock ticker symbol.
    folder (str): Folder path to save the CSV.
    
    Returns:
    None
    """
    filename = f"../{folder}/processed/{ticker}_news.csv"
    data.to_csv(filename)
    print(f"Data saved to {filename}")

In [45]:
def run_workflow(ticker, start_date, end_date):
    
    news_data = fetch_news_data(ticker, start_date, end_date, limit=500, api_key=api_key, max_articles=500)
    
    if news_data:
        save_to_csv(pd.DataFrame(news_data), ticker, folder='data')
    
    

In [46]:
# Define tickers (example for S&P 500 or other stocks)
ticker = 'SPY.US'  # Use the normalized ticker for S&P 500 (example: SPX for S&P 500)
start_date = '2010-01-01'
end_date = '2024-09-10'

run_workflow(ticker, start_date, end_date)

Data saved to ../data/processed/SPY.US_news.csv
